In [89]:
import pandas as pd
import os
import matplotlib as plt

In [90]:
files = os.listdir('./')
csv_files = [f for f in files if "0.csv" in f]
csv_files

['signals.2018-08-17-222339.0.csv', 'signals.2018-08-18-063826.0.csv']

In [91]:
frame = pd.DataFrame()
list_ = []
for file_ in csv_files:
    df = pd.read_csv(file_,index_col=None, header=None)
    list_.append(df)
frame = pd.concat(list_)
frame.columns = ["ticker", "price", "time", 
                "meta_signal", "previous_meta",
                "4h","1d","1w","1m",
                "4hp","1dp","1wp","1mp"]
frame.sort_values(['ticker', 'time'], inplace=True)
frame.head(5)

,ticker,price,time,meta_signal,previous_meta,4h,1d,1w,1m,4hp,1dp,1wp,1mp
17,AMEX-BTG,2.16,1534571497681,-1,0,-1,-1,-1,-1,0,0,0,0
17,AMEX-BTG,2.16,1534604009032,-1,0,-1,-1,-1,-1,0,0,0,0
29,AMEX-COPX,20.96,1534571773235,-1,0,-1,-1,-1,-1,0,0,0,0
29,AMEX-COPX,20.96,1534604312033,-1,0,-1,-1,-1,-1,0,0,0,0
35,AMEX-DGRO,36.73,1534571911008,2,0,1,1,2,2,0,0,0,0


In [92]:
frame.shape

(308, 13)

In [93]:
# failures to get price (numeric conversion)
frame[frame.price.isnull()]

,ticker,price,time,meta_signal,previous_meta,4h,1d,1w,1m,4hp,1dp,1wp,1mp
146,NASDAQ-VYMI,NaN,1534607161567,-1,0,-1,-2,-1,-1,-2,0,0,0
45,NYSE-EXG,NaN,1534604715456,2,0,2,1,2,1,1,0,0,0


In [94]:
frame = frame.dropna()
frame.shape

(306, 13)

In [95]:
# more failures to get price
frame[frame.price == 0]

,ticker,price,time,meta_signal,previous_meta,4h,1d,1w,1m,4hp,1dp,1wp,1mp
63,AMEX-GTE,0.0,1534605165732,0,0,-1,-1,0,1,0,0,0,0
57,NASDAQ-GERN,0.0,1534605014514,1,0,1,1,1,1,0,0,0,0


In [96]:
frame = frame[frame.price!=0]
frame.shape

(304, 13)

In [97]:
abbv = frame[frame.ticker.astype(str).str.match("NYSE-ABBV")]
abbv

,ticker,price,time,meta_signal,previous_meta,4h,1d,1w,1m,4hp,1dp,1wp,1mp
1,NYSE-ABBV,98.81,1534571111519,1,0,1,2,1,1,0,0,0,0
1,NYSE-ABBV,98.81,1534603601535,1,0,1,2,1,1,0,0,0,0


In [98]:
# only get records where something changed (price, or indicators)
df = frame.drop_duplicates(["ticker", "price", 
                "meta_signal", "previous_meta",
                "4h","1d","1w","1m",
                "4hp","1dp","1wp","1mp"])
df[df.ticker.astype(str).str.match("NYSE-ABBV")]

,ticker,price,time,meta_signal,previous_meta,4h,1d,1w,1m,4hp,1dp,1wp,1mp
1,NYSE-ABBV,98.81,1534571111519,1,0,1,2,1,1,0,0,0,0


In [99]:
# test that we have expected number of unique tickers (at least one entry for each)
# currently, we have 155 tickers
df['ticker'].nunique()

155

In [100]:
# export new, cleaned db file
df.to_csv('./signals.csv', header=None, index=None)